In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

[[Home](/)]
[[Webapps](/jupyter/voila/tree/webapps)]

# System overview



In [6]:
from stratosphere import Stratosphere
from stratosphere.services.extractor import get_list_extractors
from stratosphere import options
from IPython.display import HTML
from markdown import markdown
import os

def first_or_null(s):
    if len(s) >0:
        return s.iloc[0]
    else:
        return "N/A"

    


In [11]:
kb = Stratosphere(options.get("db.url_kb"))
probe = Stratosphere(options.get("db.url_probe"))
count_entities = kb.db.pandas("SELECT count(*) as cnt FROM entities").cnt.iloc[0]
count_relationships = kb.db.pandas("SELECT count(*) as cnt FROM relationships").cnt.iloc[0]
last_entity_ts = kb.db.pandas("SELECT ts FROM entities ORDER BY ts DESC LIMIT 1").ts.iloc[0]
last_relationship_ts = kb.db.pandas("SELECT ts FROM relationships ORDER BY ts DESC LIMIT 1").ts.iloc[0]
last_flow_ts = first_or_null(probe.db.pandas("SELECT flow_capture_timestamp FROM flows ORDER BY flow_capture_timestamp DESC LIMIT 1").flow_capture_timestamp)
count_flows = kb.db.pandas("SELECT count(*) as cnt FROM flows").cnt.iloc[0]
size_mb_probe = probe.db.size()
size_mb_kb = kb.db.size()

registered_extractors = ""
for modname in get_list_extractors():
    registered_extractors += f"* **{modname}**\n"

mkd = f"""
Overview:

* The knowledge base contains **{count_entities}** entities and **{count_relationships}** relationships
* Timestamp of last inserted entity: **{last_entity_ts}**
* Timestamp of last inserted relationship: **{last_relationship_ts}**
* Timestamp of last intercepted flow: **{last_flow_ts}**
* Count of (not deleted) flows in probe.db: **{count_flows}**
* Size (MiB) of probe.db: **{size_mb_probe:.2f}**
* Size (MiB) of kb.db: **{size_mb_kb:.2f}**

Registered extractors:

{registered_extractors}
"""

HTML(markdown(mkd))

Recent system logs:


In [12]:
!tail -100 /tmp/supervisord.log

2023-04-14 08:02:39,426 INFO Set uid to user 0 succeeded
2023-04-14 08:02:39,428 INFO RPC interface 'supervisor' initialized
2023-04-14 08:02:39,428 CRIT Server 'unix_http_server' running without any HTTP authentication checking
2023-04-14 08:02:39,428 INFO supervisord started with pid 1
2023-04-14 08:02:40,451 INFO spawned: 'extractor' with pid 7
2023-04-14 08:02:40,455 INFO spawned: 'jupyter' with pid 8
2023-04-14 08:02:40,465 INFO spawned: 'mitmproxy' with pid 9
2023-04-14 08:02:40,476 INFO spawned: 'nginx' with pid 10
2023-04-14 08:02:40,489 INFO spawned: 'sqliteweb' with pid 11
2023-04-14 08:02:42,409 INFO success: extractor entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2023-04-14 08:02:42,409 INFO success: jupyter entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2023-04-14 08:02:42,410 INFO success: mitmproxy entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2023-04-14 08:02:42,410 INFO succe

In [14]:
!echo "Uptime (hh:mm:ss): $(ps -o etime= -p 1 | tr -d ' ')"

Uptime (hh:mm:ss): 05:16
